In [1]:
import warnings

warnings.filterwarnings("ignore")

import os
import pyproj
import pandas as pd
import xarray as xr
import geopandas as gpd
import rasterio as rio
from datetime import datetime
from shapely.geometry import box, Polygon
import matplotlib.pyplot as plt

In [2]:
# Seting up the paths

boundary_path = "./AgriGeoSpatial/data/445756/BOUNDARY1028-Rt49_Lamkey_360.zip"
yield_data_path = "./AgriGeoSpatial/data/445756/Equipment_Harvest/2022-11-07/CKGK9AFQ9/harvest/116_master_debug.zip"

### Step 01:

    a. Load Boundary data
    b. cut the boundary geometry into 20 m x 20 m box


In [3]:
def subset_geom(
    path: str, crs: pyproj.crs.crs.CRS = "epsg:32615", grid_resolution: int = 20
):
    """generate resolution grid_res x grid_rs

    Parameters
    ----------
        :path
        :crs

    Return
    ------

    """
    # Load the two GeoDataFrames
    gpd_df = gpd.read_file(path)
    original_crs = gpd_df.crs

    # convert degrees to meter
    gpd_df = gpd_df.to_crs(crs)

    # Get the bounding box of the shapefile
    xmin, ymin, xmax, ymax = gpd_df.total_bounds

    # Calculate the number of rows and columns in the grid based on the resolution
    num_cols = int((xmax - xmin) / grid_resolution)
    num_rows = int((ymax - ymin) / grid_resolution)

    # Generate a grid of polygons within the bounding box
    polygons = []
    for row in range(num_rows + 2):
        for col in range(num_cols + 2):
            x1 = xmin + col * grid_resolution
            y1 = ymin + row * grid_resolution
            x2 = x1 + grid_resolution
            y2 = y1 + grid_resolution
            polygons.append(box(x1, y1, x2, y2))

    # Create a GeoDataFrame from the list of polygons
    return gpd.GeoDataFrame(geometry=polygons, crs=crs).to_crs(original_crs)

In [4]:
# boundary_path = './AgriGeoSpatial/data/445863/BOUNDARY1016-Rt49_Williams_Big.zip'
gdf1 = subset_geom(boundary_path)

### Step 02

    a. load yield data
    b. get geometry from overlap - intersection
    c. count each geometry area
    d. set filter to filter out the area is larger than area threshold, which is based on **area_mean - area_std**.


In [5]:
def clip_with_harvest(geom, yield_path, MinAreaPercentage=0.6, TotalArea=400.0):
    """
    Argument
    --------
     :geom: boxes of geometry
     :yield_path: yield data path
     :MinAreaPercentage: an area ratio of minimum selection

    Return
    ------
     :normally-spaced yield data with eliminated boxes
    """
    # - Step 01: Load Yield field
    yield_df = gpd.read_file(yield_path)

    # - Step 02: generate a empty list to store the outputs
    valid_geom = []

    # - Step 03: Loop for each box geom
    # -    (a) eliminated if total area of yield data is less than 0.6 of total area
    # -    (b) spatial-weighted average
    for num, (idx, row) in enumerate(geom.iterrows()):
        # - make geometry dataframe by box
        row_gdf = gpd.GeoDataFrame(
            row.to_frame().T, geometry="geometry", crs=yield_df.crs
        )

        # - clipped yield in box
        clipped = gpd.clip(yield_df, row_gdf)
        clipped["clipped_area"] = clipped.to_crs("epsg:32615").area

        # - spatial-weighted average if overlapped area > 0.6 of total area
        if clipped["clipped_area"].sum() / TotalArea > MinAreaPercentage:
            valid_geom.append(spatial_normalized_average(row_gdf, clipped))

    if len(valid_geom) == 0:
        raise ValueError(
            {
                "status": "failed",
                "reason": "no overlap, please check the inputs are correct.",
            }
        )

    valid_geom_df = pd.concat(valid_geom)
    return gpd.GeoDataFrame(
        valid_geom_df.reset_index().drop(columns=["index"], axis=1),
        geometry="geometry",
        crs=yield_df.crs,
    )


def spatial_normalized_average(row, clipped):
    """
    Argument
    --------
       row: 20 m x 20 m box geopandas dataframe
       clipped: clipped yield dataframe

    Return
    ------
       normally-spaced yield data

    """
    selected_clipped = clipped.select_dtypes(include=[int, float]).iloc[:, :-1]
    selected_clipped_average = (
        selected_clipped.multiply(clipped["clipped_area"], axis="index").sum()
        / clipped["clipped_area"].sum()
    )
    return selected_clipped_average.to_frame().T.assign(
        geometry=row["geometry"].iloc[0]
    )

In [6]:
# yield_data_path = './AgriGeoSpatial/data/445863/Equipment_Harvest/2022-10-22/343PMT4T6/harvest/296_master_debug.zip'
valid_geometry_df = clip_with_harvest(gdf1, yield_data_path)

In [7]:
valid_geometry_df

### Shadow detection on Aeroptic Dataset.

Functions


In [8]:
import rioxarray as rx
import numpy as np


def ndvi(nir, red):
    return (nir - red) / (nir + red)


def gndvi(nir, green):
    return (nir - green) / (nir + green)


def masked_bare_soil(red_band, green_band, images="Aeroptic"):
    masked = green_band / red_band
    threshold = (
        1.35
        if images == "Spot"
        else 1.2
        if images == "Aeroptic"
        else 1.15
        if images == "Pleiades"
        else None
    )
    masked = green_band / red_band
    return xr.where(masked >= threshold, 1, 0)

In [9]:
from typing import Dict


def image_processed(
    valid_geometry_df: gpd.GeoDataFrame, image: str, image_path: str, bands: Dict
) -> gpd.GeoDataFrame:
    """derive the relative values on RGB+NIR bands + NDVI + GNDVI, and average values of them
          reference values is 10th percentile on RGB, 90th percentile on NIR, NDVI and GNDVI

    Argument
    --------
        :valid_geometry_df, geometry of boxes
        :image, image name
        :image_path, data path of remote sensing images
        :bands, dict of RGBN, ex: {'R':1'}

    Return
    ------
       return the original dataframe with image values
    """
    # - Step 01: Load Image
    image_df = rx.open_rasterio(image_path)

    # - Step 02: Generate a Mask using bare soil
    masked = masked_bare_soil(
        image_df.sel(band=bands["R"]), image_df.sel(band=bands["G"]), images=image
    )

    # - Step 03: Generate NDVI and GNDVI
    NDVI = ndvi(
        image_df.sel(band=bands["N"]).where(image_df["masked" == 1]),
        image_df.sel(band=bands["R"]).where(image_df["masked" == 1]),
    )

    GNDVI = gndvi(
        image_df.sel(band=bands["N"]).where(image_df["masked" == 1]),
        image_df.sel(band=bands["G"]).where(image_df["masked" == 1]),
    )

    # - Step 04: 10th percential on RGB bands
    REDp10 = np.nanpercentile(
        xr.where(masked == 1, image_df.sel(band=bands["R"]), np.nan), 10
    )
    GRNp10 = np.nanpercentile(
        xr.where(masked == 1, image_df.sel(band=bands["G"]), np.nan), 10
    )
    BLUp10 = np.nanpercentile(
        xr.where(masked == 1, image_df.sel(band=bands["B"]), np.nan), 10
    )

    # - Step 05: 90th percential on NIR, NDVI, GNDVI
    NIRp90 = np.nanpercentile(
        xr.where(masked == 1, image_df.sel(band=bands["N"]), np.nan), 90
    )
    NDVIp90 = np.nanpercentile(xr.where(masked == 1, NDVI, np.nan), 90)
    GNDVIp90 = np.nanpercentile(xr.where(masked == 1, GNDVI, np.nan), 90)

    print(
        f"{image}  RGBN NDVI GNDVI percentile: ",
        REDp10,
        GRNp10,
        BLUp10,
        NIRp90,
        NDVIp90,
        GNDVIp90,
    )
    print(f"{image}  GNDVI: ", GNDVI.min().values, GNDVI.max().values)
    print(f"{image}  NDVI: ", NDVI.min().values, NDVI.max().values, "\n\n")

    # - Step 06: Clip and Generate the final outputs
    image_trimmed_df = []
    for idx, row in valid_geometry_df.iterrows():
        row_gdf = gpd.GeoDataFrame(
            row.to_frame().T, geometry="geometry", crs=valid_geometry_df.crs
        ).to_crs(image_df.rio.crs)

        clipped = image_df.rio.clip([row_gdf.geometry.item()])
        clipped_mask = masked.rio.clip([row_gdf.geometry.item()])
        clipped_NDVI = NDVI.rio.clip([row_gdf.geometry.item()])
        clipped_GNDVI = GNDVI.rio.clip([row_gdf.geometry.item()])

        NIR, RED = clipped.sel(band=bands["N"]), clipped.sel(band=bands["R"])
        GRN, BLU = clipped.sel(band=bands["G"]), clipped.sel(band=bands["B"])

        # - Relative value in box
        RlvRED = xr.where(clipped_mask == 1, RED / REDp10, np.nan).mean().values
        RlvGRN = xr.where(clipped_mask == 1, GRN / GRNp10, np.nan).mean().values
        RlvBLU = xr.where(clipped_mask == 1, BLU / BLUp10, np.nan).mean().values
        RlvNIR = xr.where(clipped_mask == 1, NIR / NIRp90, np.nan).mean().values
        RlvNDVI = (
            xr.where(clipped_mask == 1, clipped_NDVI / NDVIp90, np.nan).mean().values
        )
        RlvGNDVI = (
            xr.where(clipped_mask == 1, clipped_GNDVI / GNDVIp90, np.nan).mean().values
        )

        # - Average in box
        AveRED = xr.where(clipped_mask == 1, RED, np.nan).mean().values
        AveGRN = xr.where(clipped_mask == 1, GRN, np.nan).mean().values
        AveBLU = xr.where(clipped_mask == 1, BLU, np.nan).mean().values
        AveNIR = xr.where(clipped_mask == 1, NIR, np.nan).mean().values
        AveNDVI = xr.where(clipped_mask == 1, clipped_NDVI, np.nan).mean().values
        AveGNDVI = xr.where(clipped_mask == 1, clipped_GNDVI, np.nan).mean().values

        row_gdf[
            [
                f"{image} Average NIR",
                f"{image} Average RED",
                f"{image} Average Blue",
                f"{image} Average Green",
                f"{image} Average NDVI",
                f"{image} Average GNDVI",
            ]
        ] = [AveNIR, AveRED, AveBLU, AveGRN, AveNDVI, AveGNDVI]

        row_gdf[
            [
                f"{image} Relative NIR",
                f"{image} Relative RED",
                f"{image} Relative Blue",
                f"{image} Relative Green",
                f"{image} Relative NDVI",
                f"{image} Relative GNDVI",
            ]
        ] = [RlvNIR, RlvRED, RlvBLU, RlvGRN, RlvNDVI, RlvGNDVI]

        image_trimmed_df.append(row_gdf)

    image_trimmed_df = pd.concat(image_trimmed_df)
    return gpd.GeoDataFrame(image_trimmed_df, geometry="geometry", crs=image_df.rio.crs)

In [10]:
Total_now = datetime.now()

bands = ["RGBN"]
bands = {char: idx + 1 for idx, char in enumerate(bands[0])}
print(bands)

now = datetime.now()

image, image_path = (
    "Aeroptic",
    "./AgriGeoSpatial/data/445756/Aeroptic/2022-06-28/HHZT8D9QA/merged_out.tif",
)
image_trimmed_df = image_processed(valid_geometry_df, image, image_path, bands)
images01_cost = datetime.now() - now

print(image_trimmed_df.columns)
image_trimmed_df.head()

{'R': 1, 'G': 2, 'B': 3, 'N': 4}
Aeroptic  RGBN NDVI GNDVI percentile:  77.0 112.0 93.0 255.0 0.5315315315315315 0.3896457765667575
Aeroptic  GNDVI:  -0.24175824175824176 0.5
Aeroptic  NDVI:  -0.007194244604316547 0.6037735849056604 




In [11]:
now = datetime.now()

image, image_path = (
    "Pleiades",
    "./AgriGeoSpatial/data/445756/Airbus_Pleiades/2022-06-28/YZ9DJEHVV/merged_out.tif",
)
image_trimmed_df = image_processed(image_trimmed_df, image, image_path, bands)
images02_cost = datetime.now() - now

print(image_trimmed_df.columns)
image_trimmed_df.head()

Pleiades  RGBN NDVI GNDVI percentile:  368.0 671.0 511.0 5011.0 0.8618194971965998 0.7595862314963439
Pleiades  GNDVI:  0.076875 0.8190196899874319
Pleiades  NDVI:  0.11276156032033066 0.8891769280622165 




In [12]:
now = datetime.now()

image, image_path = (
    "Spot",
    "./AgriGeoSpatial/data/445756/Airbus_Spot/2022-06-27/WJYIEGKB8/merged_out.tif",
)
image_trimmed_df = image_processed(image_trimmed_df, image, image_path, bands)
images03_cost = datetime.now() - now

print(image_trimmed_df.columns)
image_trimmed_df.head()

Spot  RGBN NDVI GNDVI percentile:  177.0 312.0 321.0 1913.0 0.824912456228114 0.7098111469368954
Spot  GNDVI:  0.20509322419281492 0.7503040129712201
Spot  NDVI:  0.22631578947368422 0.8530612244897959 




In [ ]:
image_trimmed_df.to_csv("445756-originalBoundary_processed.csv")